# OWL and Blazegraph 
This programme is to practice access to Blazegraph from python. It creates a sample ontology and tries to upload it to BG and then conduct a SPARQL query. 

In [4]:
# %pip install owlready2
# %pip install requests

# import libraries
from owlready2 import *
from rdflib import Graph, Namespace
from rdflib.namespace import OWL, RDF, RDFS
import requests


In [5]:

# # Create a graph
# kg = Graph()

# # Define namespaces
# ex = Namespace("http://example.org/")
# kg.bind("ex", ex)
# kg.bind("owl", OWL)

# # Define OWL classes
# kg.add((ex.Person, RDF.type, OWL.Class))
# kg.add((ex.Organization, RDF.type, OWL.Class))
# kg.add((ex.Location, RDF.type, OWL.Class))

# # Define OWL properties
# kg.add((ex.works_at, RDF.type, OWL.ObjectProperty))
# kg.add((ex.location, RDF.type, OWL.ObjectProperty))

# # Add individuals and relationships
# kg.add((ex.Alice, RDF.type, ex.Person))
# kg.add((ex.Bob, RDF.type, ex.Person))
# kg.add((ex.Markus, RDF.type, ex.Person))
# kg.add((ex.Google, RDF.type, ex.Organization))
# kg.add((ex.Microsoft, RDF.type, ex.Organization))
# kg.add((ex.MIT, RDF.type, ex.Organization))
# kg.add((ex.California, RDF.type, ex.Location))
# kg.add((ex.Washington, RDF.type, ex.Location))
# kg.add((ex.Cambridge, RDF.type, ex.Location))

# kg.add((ex.Alice, ex.works_at, ex.Google))
# kg.add((ex.Bob, ex.works_at, ex.Microsoft))
# kg.add((ex.Markus, ex.works_at, ex.MIT))
# kg.add((ex.Google, ex.location, ex.California))
# kg.add((ex.Microsoft, ex.location, ex.Washington))
# kg.add((ex.MIT, ex.location, ex.Cambridge))

In [6]:
# # Serialize the graph to an OWL file
# kg.serialize("my_knowledge_graph.owl", format="xml")


In [10]:
# Create a new ontology with a given IRI
onto = get_ontology("http://example.org/my_BG_ontology.owl")

with onto:
    # Define the main classes
    class Person(Thing):
        pass

    class Organization(Thing):
        pass

    class Location(Thing):
        pass

    # Define object properties
    class works_at(ObjectProperty):
        domain = [Person]
        range = [Organization]

    class location(ObjectProperty):
        domain = [Organization]
        range = [Location]

    # Create individual instances
    alice = Person("Alice")
    bob = Person("Bob")
    markus = Person("Markus")

    google = Organization("Google")
    microsoft = Organization("Microsoft")
    mit = Organization("MIT")

    california = Location("California")
    washington = Location("Washington")
    cambridge = Location("Cambridge")

    # Define the relationships
    alice.works_at.append(google)
    bob.works_at.append(microsoft)
    markus.works_at.append(mit)

    google.location.append(california)
    microsoft.location.append(washington)
    mit.location.append(cambridge)

# Save the ontology to a file
onto.save(file="my_BG_ontology.owl", format="rdfxml")

In [12]:
# # Blazegraph endpoint - can only read
# blazegraph_url = "http://localhost:3838/blazegraph/namespace/kb/sparql"

# Blazegraph endpoint can also write
blazegraph_url = "http://localhost:3838/blazegraph/namespace/kb/update"


# Path to your RDF file
rdf_file_path = r"C:\VS mk306 TestEnv\Codes\mk_PracticeEnv\my_BG_ontology.owl"


# Headers for the request
# headers = {
#     "Content-Type": "text/turtle"
# }
headers = {
    "Content-Type": "application/rdf+xml"
}

# Read the RDF file
with open(rdf_file_path, "r") as file:
    rdf_data = file.read()

# Send the data to Blazegraph
response = requests.post(blazegraph_url, headers=headers, data=rdf_data)

# Check the response
if response.status_code == 200:
    print("Data loaded successfully!")
else:
    print(f"Failed to load data. Status code: {response.status_code}")
    print(response.text)




Data loaded successfully!


In [13]:
# # Blazegraph endpoint
# blazegraph_url = "http://localhost:3838/blazegraph/namespace/kb/sparql"


# Blazegraph SPARQL Query Endpoint
sparql_endpoint = blazegraph_url

# # SPARQL Query  1
# sparql_query = """
# SELECT ?s ?p ?o
# WHERE {
#     ?s ?p ?o .
# } LIMIT 5
# """

# # Send HTTP request
# response = requests.get(sparql_endpoint, params={"query": sparql_query, "format": "json"})

# # Parse JSON response
# results = response.json()

# # Print results
# for result in results["results"]["bindings"]:
#     print(result["s"]["value"], result["p"]["value"], result["o"]["value"])


# # SPARQL Query  2
# sparql_query = """
# PREFIX ex: <http://example.org/my_BG_ontology.owl#>
# SELECT ?organization ?location
# WHERE {
#   ?organization a ex:Organization .
#   ?organization ex:location ?location .
# }
# """

# # SPARQL endpoint URL
# sparql_endpoint = blazegraph_url  # Replace with your actual SPARQL endpoint

# # Send HTTP request
# response = requests.get(sparql_endpoint, params={"query": sparql_query, "format": "json"})

# # Check if the request was successful
# if response.status_code == 200:
#     # Parse JSON response
#     results = response.json()

#     # Print results
#     for result in results["results"]["bindings"]:
#         organization = result["organization"]["value"]
#         location = result["location"]["value"]
#         print(f"Organization: {organization}, Location: {location}")
# else:
#     print(f"Error: {response.status_code} - {response.text}")


# SPARQL Query  3
sparql_query = """
PREFIX ex: <http://example.org/my_BG_ontology.owl#>
SELECT ?person ?organization
WHERE {
  ?person a ex:Person .
  ?person ex:works_at ?organization .
}
"""

# SPARQL endpoint URL
sparql_endpoint = blazegraph_url  # Replace with your actual SPARQL endpoint

# Send HTTP request
response = requests.get(sparql_endpoint, params={"query": sparql_query, "format": "json"})

# Check if the request was successful
if response.status_code == 200:
    # Parse JSON response
    results = response.json()

    print(results) # {'head': {'vars': ['person', 'organization']}, 'results': {'bindings': []}}

    # Print results
    for result in results["results"]["bindings"]:
        organization = result["organization"]["value"]
        person = result["person"]["value"]
        print(f"Organization: {organization}, Person: {person}")
else:
    print(f"Error: {response.status_code} - {response.text}")



{'head': {'vars': ['person', 'organization']}, 'results': {'bindings': [{'person': {'type': 'uri', 'value': 'http://example.org/my_BG_ontology.owl#Alice'}, 'organization': {'type': 'uri', 'value': 'http://example.org/my_BG_ontology.owl#Google'}}, {'person': {'type': 'uri', 'value': 'http://example.org/my_BG_ontology.owl#Bob'}, 'organization': {'type': 'uri', 'value': 'http://example.org/my_BG_ontology.owl#Microsoft'}}, {'person': {'type': 'uri', 'value': 'http://example.org/my_BG_ontology.owl#Markus'}, 'organization': {'type': 'uri', 'value': 'http://example.org/my_BG_ontology.owl#MIT'}}]}}
Organization: http://example.org/my_BG_ontology.owl#Google, Person: http://example.org/my_BG_ontology.owl#Alice
Organization: http://example.org/my_BG_ontology.owl#Microsoft, Person: http://example.org/my_BG_ontology.owl#Bob
Organization: http://example.org/my_BG_ontology.owl#MIT, Person: http://example.org/my_BG_ontology.owl#Markus
